In [1]:
import numpy as np
import numpy.matlib
import math
import random
from scipy.stats import norm
from scipy.stats import uniform


In [2]:
#Create column vectors for initial share values 
S0 = np.array([[100],[95],[50]])
#Volatilities for each of the shares
sigma = np.array([[0.15],[0.2],[0.3]])
#Correlation
cor_matrix = np.array([[1,0.2,0.4],[0.2,1,0.8],[0.4,0.8,1]])
L = np.linalg.cholesky(cor_matrix)
#risk free rate
r = 0.1
#Time period 
T = 1

In [3]:
np.random.seed(0)
t_simulations = 10000
#Confidence interval
alpha = 0.05
#Current portfolio value 
portfolio_cur = np.sum(S0)

In [4]:
def find_terminal_price(S0,r,sigma,T,Z):
    return S0 * np.exp((r - sigma**2/2) * T + sigma * np.sqrt(T) * Z)

In [5]:
#Assuming holdings is constant simulate future portfolio values
Z = np.matmul(L,norm.rvs(size = [3,t_simulations]))
portfolio_fut = np.sum(find_terminal_price(S0,r,sigma,T,Z),axis = 0)


In [6]:
#Calculating portfolio returns 
portfolio_returns = (portfolio_fut - portfolio_cur) / portfolio_cur
#Sorting
portfolio_returns = np.sort(portfolio_returns)
#Calculating VaR for portfolio
#VaR is the negative of the alpha quantile , alpha = 5%
var_estimate = -portfolio_returns[int(np.floor(alpha * t_simulations)) - 1]
print("VaR estimate with Monte Carlo : {} %".format(var_estimate * 100))

VaR estimate with Monte Carlo : 15.694930298355253 %


In [7]:
#Historical data simulation
random.seed(0)
t_simulations = 10000
alpha = 0.05

In [8]:
#Instead of using historical data, lets generate our own
#Function would be the same as the previous terminal price function but use cumulative sum to keep tally of daily returns
def gen_share_path(S0,r,sigma,T,Z):
    return S0 * np.exp(np.cumsum((r - sigma**2/2) * T + sigma * np.sqrt(T) * Z,1))


In [9]:
# 3 Random Variables for 3 different assets in the portfolio for 5 years
Z = norm.rvs(size = [3,5*365])
corr_Z = (np.matmul(L,Z))


In [10]:
price_path = gen_share_path(S0,r,sigma,T,corr_Z)

In [12]:
hist_S0 = price_path[-1]
portfolio_history = np.sum(hist_S0)
portfolio_returns = [None] * t_simulations
#Historical log returns
portfolio_log_returns = np.log(price_path[1:]) - np.log(price_path[0:-1])

In [15]:
for i in range(t_simulations):
    random_sample = uniform.rvs(size = 365) * (len(price_path) - 1)
    random_sample = [int(x) for x in random_sample]
    share_returns = portfolio_log_returns[random_sample]
    s_term = hist_S0 * np.exp(np.sum(share_returns,axis = 0))
    portfolio_returns[i] = (np.sum(s_term) - portfolio_history) / portfolio_history

In [22]:
#Sort
portfolio_returns = np.sort(portfolio_returns)

#VaR estimate
hVar_estimate = -portfolio_returns[int(np.floor(alpha*t_simulations)) - 1]
print("Historical data VaR estimate : {}".format(hVar_estimate))

Historical data VaR estimate : -1.2273746945959654e+98
